## Introduction to Translation with Transformers and MLflow

In this tutorial, we delve into the world of language translation by leveraging the power of [Transformers](https://huggingface.co/docs/transformers) and MLflow. This guide is crafted for practitioners with a grasp of machine learning concepts who seek to streamline their translation model workflows. We will showcase the use of MLflow to log, manage, and serve a cutting-edge translation model - the `google/flan-t5-base` from the [🤗 Hugging Face](https://huggingface.co/) library.

### Learning Objectives

Throughout this tutorial, you will:

- Construct a translation **pipeline** using `flan-t5-base` from the Transformers library.
- **Log** the translation model and its configurations using MLflow.
- Determine the input and output **signature** of the translation model automatically.
- **Retrieve** a logged translation model from MLflow for direct interaction.
- Emulate the deployment of the translation model using MLflow's **pyfunc** model flavor for language translation tasks.

By the conclusion of this tutorial, you'll gain a thorough insight into managing and deploying translation models with MLflow, thereby enhancing your machine learning operations for language processing.

To learn more, expand the **details** sections throughout this tutorial.

<details>
    <div>
        <h3>Why was this model chosen?</h3>
        <p>The <a href="https://huggingface.co/google/flan-t5-base">flan-t5-base</a> offers a few benefits:</p>
        <ul>
            <li><strong>Size</strong>: it's a relatively small model for the comparatively powerful performance.</li>
            <li><strong>Enhanced Language Coverage</strong>: Expanding on the original <a href="https://huggingface.co/t5-base">T5 model</a>, the flan-t5 has a much larger breadth of languages that it supports.</li>
        </ul>
    </div>
</details>

### Setting Up the Translation Environment

Begin by setting up the essential components for translation tasks using the google/flan-t5-base model.

<details>
    <div>
        <h4>Importing Libraries</h4>
        <p>We import the `transformers` library for access to the translation model and tokenizer. Additionally, `mlflow` is included for model tracking and management, creating a comprehensive environment for our translation tasks.</p>
    </div>
    <div>
        <h4>Initializing the Model</h4>
        <p>The `google/flan-t5-base` model, known for its translation effectiveness, is loaded from the Hugging Face repository. This pre-trained model is a key component of our setup.</p>
    </div>
    <div>
        <h4>Setting Up the Tokenizer</h4>
        <p>We initialize the tokenizer corresponding to our model. The tokenizer plays a critical role in processing text input, making it understandable for the model.</p>
    </div>
    <div>
        <h4>Creating the Pipeline</h4>
        <p>A translation pipeline for English to French is created. This pipeline streamlines the process, allowing us to focus on inputting text and receiving translations without managing model and tokenizer interactions directly.</p>
    </div>
</details>
<br/>

In [1]:
# Disable tokenizers warnings when constructing pipelines
%env TOKENIZERS_PARALLELISM=false

import warnings

# Disable a few less-than-useful UserWarnings from setuptools and pydantic
warnings.filterwarnings("ignore", category=UserWarning)

env: TOKENIZERS_PARALLELISM=false


In [2]:
import transformers

import mlflow

model_architecture = "google/flan-t5-base"

translation_pipeline = transformers.pipeline(
    task="translation_en_to_fr",
    model=transformers.T5ForConditionalGeneration.from_pretrained(
        model_architecture, max_length=1000
    ),
    tokenizer=transformers.T5TokenizerFast.from_pretrained(model_architecture, return_tensors="pt"),
)

### Testing the Translation Pipeline

We perform a preliminary check on our translation pipeline to ensure its proper functioning before logging it with MLflow.

<details>
    <div>
        <h4>Model Verification</h4>
        <p>A test translation allows us to verify that the model accurately translates text, in this case from English to French, ensuring the model's basic functionality.</p>
    </div>
    <div>
        <h4>Error Prevention</h4>
        <p>Identifying potential issues before model logging helps prevent future errors during deployment or inference, saving time and resources.</p>
    </div>
    <div>
        <h4>Resource Management</h4>
        <p>Testing minimizes wasteful use of resources, particularly important given the large size of these models and the resources needed to save and load them.</p>
    </div>
    <div>
        <h4>Pipeline Validation</h4>
        <p>This step confirms that both the model and tokenizer in the pipeline are correctly configured and capable of processing the input as expected.</p>
    </div>
</details>
<br/>

In [3]:
# Evaluate the pipeline on a sample sentence prior to logging
translation_pipeline(
    "translate English to French: I enjoyed my slow saunter along the Champs-Élysées."
)

[{'translation_text': "J'ai apprécié mon sajour lente sur les Champs-Élysées."}]

### Evaluating the Translation Results

Upon running our initial translation through the pipeline, we observed that the output, while generally accurate, exhibited areas for improvement.

<details>
    <div>
        <p>The initial translation output was:</p>
        <blockquote>
            <p><code>[{'translation_text': "J'ai apprécié mon sajour lente sur les Champs-Élysées."}]</code></p>
        </blockquote>
        <p>This translation captures the essence of the original English sentence but shows minor grammatical errors and word choice issues. For instance, a more refined translation might be:</p>
        <blockquote>
            <p><code>"J'ai apprécié ma lente promenade le long des Champs-Élysées."</code></p>
        </blockquote>
        <p>This version corrects grammatical gender and adds necessary articles, accentuation, and hyphenation. These subtle nuances enhance the translation quality significantly.</p>
        <p>The base model's performance is encouraging, indicating the potential for more precise translations with further fine-tuning and context. MLflow's tracking and management capabilities will be instrumental in monitoring the iterative improvements of such models.</p>
        <p>In summary, while the pursuit of perfection in machine translation is ongoing, the initial results are a promising step towards achieving natural and accurate translations.</p>
    </div>
</details>

### Setting Model Parameters and Inferring Signature

We establish crucial model parameters and infer the signature to ensure consistency and reliability in our model's deployment.

<details>
    <div>
        <h4>Defining Model Parameters</h4>
        <p>Setting key parameters like <code>max_length</code> is vital for controlling model behavior during inference. For example, a <code>max_length</code> of 1000 ensures the model handles longer sentences effectively, crucial for maintaining context in translations.</p>
    </div>
    <div>
        <h4>Importance of Inferring Signature</h4>
        <p>The signature, defining the model's input and output schema, is critical for MLflow's understanding of the expected data structures. By inferring this signature, we document the types and structures of data that the model works with, enhancing its reliability and portability.</p>
    </div>
    <div>
        <h4>Benefits of This Process</h4>
        <ul>
            <li><strong>Enhanced Portability</strong>: Properly defined parameters and signatures make the model more adaptable to different environments.</li>
            <li><strong>Error Reduction</strong>: This step minimizes the risk of encountering schema-related errors during deployment.</li>
            <li><strong>Clear Documentation</strong>: It serves as a clear guide for developers and users, simplifying the model's integration into applications.</li>
        </ul>
    </div>
    <div>
        <p>By establishing these parameters and signature, we lay a robust foundation for our model's subsequent tracking, management, and serving via MLflow.</p>
    </div>
</details>
<br/>

In [4]:
# Define the parameters that we are permitting to be used at inference time, along with their default values if not overridden
model_params = {"max_length": 1000}

# Generate the model signature by providing an input, the expected output, and (optionally), parameters available for overriding at inference time
signature = mlflow.models.infer_signature(
    "This is a sample input sentence.",
    mlflow.transformers.generate_signature_output(translation_pipeline, "This is another sample."),
    params=model_params,
)

### Reviewing the Model Signature

After configuring the translation model and inferring its signature, it's crucial to review the signature to confirm it matches our model's input and output structures.

<details>
    <div>
        <p>The model signature serves as a blueprint for MLflow to interact with the model, encompassing:</p>
        <ul>
            <li><strong>Inputs:</strong> The expected input types, such as a string for the text to be translated.</li>
            <li><strong>Outputs:</strong> The output data types, which in our case is a string representing the translated text.</li>
            <li><strong>Parameters:</strong> Additional configurable settings like `max_length`, determining the maximum length of the translation output.</li>
        </ul>
        <p>Reviewing the signature through the `signature` command allows us to validate the data formats and ensure that our model will function as expected when deployed. This step is vital for consistent model performance and avoiding errors in a production environment.</p>
        <p>Furthermore, the inclusion of parameters in the signature with default values ensures that any modifications during inference are deliberate and well-documented, contributing to the model's predictability and transparency.</p>
    </div>
</details>

In [5]:
# Visualize the model signature
signature

inputs: 
  [string]
outputs: 
  [string]
params: 
  ['max_length': long (default: 1000)]

### Creating an experiment

We create a new MLflow Experiment so that the run we're going to log our model to does not log to the default experiment and instead has its own contextually relevant entry.

In [6]:
# If you are running this tutorial in local mode, leave the next line commented out.
# Otherwise, uncomment the following line and set your tracking uri to your local or remote tracking server.

# mlflow.set_tracking_uri("http://127.0.0.1:8080")

mlflow.set_experiment("Translation")

<Experiment: artifact_location='file:///Users/benjamin.wilson/repos/mlflow-fork/mlflow/docs/source/llms/transformers/tutorials/translation/mlruns/996217394074032926', creation_time=1701286351921, experiment_id='996217394074032926', last_update_time=1701286351921, lifecycle_stage='active', name='Translation', tags={}>

### Logging the Model with MLflow

We are now set to log our translation model with MLflow, ensuring its trackability and version control.

<details>
    <div>
        <h4>Starting an MLflow Run</h4>
        <p>We initiate the logging process by starting an MLflow run. This encapsulates all the model information, including artifacts and parameters, within a unique run ID.</p>
    </div>
    <div>
        <h4>Using <code>mlflow.transformers.log_model</code></h4>
        <p>This function is integral to logging our model in MLflow. It records various aspects of the model:</p>
        <ul>
            <li><strong>Model Pipeline</strong>: The complete translation model pipeline, encompassing the model and tokenizer.</li>
            <li><strong>Artifact Path</strong>: The directory path in the MLflow run where the model artifacts are stored.</li>
            <li><strong>Model Signature</strong>: The pre-defined signature indicating the model's expected input-output formats.</li>
            <li><strong>Model Parameters</strong>: Key parameters of the model, like <code>max_length</code>, providing insights into model behavior.</li>
        </ul>
    </div>
    <div>
        <h4>Outcome of Model Logging</h4>
        <p>Post logging, we obtain the <code>model_info</code> object, which encompasses all the essential metadata about the logged model, such as its storage location. This metadata is vital for future deployment and performance analysis.</p>
    </div>
</details>
<br/>

In [7]:
with mlflow.start_run():
    model_info = mlflow.transformers.log_model(
        transformers_model=translation_pipeline,
        artifact_path="french_translator",
        signature=signature,
        model_params=model_params,
    )

### Inspecting the Loaded Model Components

After loading the model from MLflow, we delve into its individual components to verify their setup and functionality.

<details>
    <div>
        <h4>Component Breakdown</h4>
        <p>The loaded model comprises several key components, each playing a crucial role in its operation:</p>
        <ul>
            <li><strong>Task</strong>: Defines the model's specific use-case, confirming its suitability for the intended task.</li>
            <li><strong>Device Map</strong>: Details the hardware configuration, important for performance optimization.</li>
            <li><strong>Model Instance</strong>: The core <code>T5ForConditionalGeneration</code> model, central to the translation process.</li>
            <li><strong>Tokenizer</strong>: The <code>T5TokenizerFast</code>, responsible for processing text inputs into a format understandable by the model.</li>
            <li><strong>Framework</strong>: Indicates the underlying deep learning framework, essential for compatibility considerations.</li>
        </ul>
    </div>
    <div>
        <h4>Ensuring Component Integrity and Functionality</h4>
        <p>Inspecting these components ensures that:</p>
        <ul>
            <li>The model aligns with our task requirements.</li>
            <li>Hardware resources are optimally utilized.</li>
            <li>Text inputs are correctly preprocessed for model consumption.</li>
            <li>The model's compatibility with the selected deep learning framework is confirmed.</li>
        </ul>
        <p>This verification step is vital for the successful application of the model in practical scenarios, reinforcing the robustness and flexibility of MLflow.</p>
    </div>
</details>
<br/>

In [8]:
# Load our saved model as a dictionary of components, comprising the model itself, the tokenizer, and any other components that were saved
translation_components = mlflow.transformers.load_model(
    model_info.model_uri, return_type="components"
)

# Show the components that made up our pipeline that we saved and what type each are
for key, value in translation_components.items():
    print(f"{key} -> {type(value).__name__}")

2023/11/30 12:00:44 INFO mlflow.transformers: 'runs:/2357c12ca17a4f328b2f72cbb7d70343/french_translator' resolved as 'file:///Users/benjamin.wilson/repos/mlflow-fork/mlflow/docs/source/llms/transformers/tutorials/translation/mlruns/996217394074032926/2357c12ca17a4f328b2f72cbb7d70343/artifacts/french_translator'


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

task -> str
device_map -> str
model -> T5ForConditionalGeneration
tokenizer -> T5TokenizerFast
framework -> str


### Understanding Model Flavors in MLflow

The `model_info.flavors` attribute in MLflow provides insights into the model's capabilities and deployment requirements across various platforms.

<details>
    <div>
        <p>Flavors in MLflow represent different ways the model can be utilized and deployed. Key aspects include:</p>
        <ul>
            <li><strong>Python Function Flavor:</strong> Indicates the model's compatibility as a generic Python function, including model binary, loader module, Python version, and environment specifications.</li>
            <li><strong>Transformers Flavor:</strong> Tailored for models from the Hugging Face Transformers library, covering transformers version, code dependencies, task, instance type, source model name, pipeline model type, framework, tokenizer type, components, and model binary.</li>
        </ul>
        <p>This information guides how to interact with the model within MLflow, ensuring proper deployment with the right environment and dependencies, whether for inference or further model refinement.</p>
    </div>
</details>

In [9]:
# Show the model parameters that were saved with our model to gain an understanding of what is recorded when saving a transformers pipeline
model_info.flavors

{'python_function': {'model_binary': 'model',
  'loader_module': 'mlflow.transformers',
  'python_version': '3.8.13',
  'env': {'conda': 'conda.yaml', 'virtualenv': 'python_env.yaml'}},
 'transformers': {'transformers_version': '4.34.1',
  'code': None,
  'task': 'translation_en_to_fr',
  'instance_type': 'TranslationPipeline',
  'source_model_name': 'google/flan-t5-base',
  'pipeline_model_type': 'T5ForConditionalGeneration',
  'framework': 'pt',
  'tokenizer_type': 'T5TokenizerFast',
  'components': ['tokenizer'],
  'model_binary': 'model'}}

### Evaluating the Translation Output

After testing our pipeline with a challenging sentence, we assess the translation's accuracy.

<details>
    <div>
        <h4>Assessing Translation Nuances</h4>
        <p>The model impressively interprets "Nice" correctly as a city name, rather than an adjective. This shows its ability to discern context and proper nouns. Furthermore, it cleverly substitutes the English play on words with the French adjective "bien," maintaining the sentence's intended sentiment.</p>
    </div>
    <div>
        <h4>Contextual Understanding</h4>
        <p>This translation exemplifies the model's strength in understanding context and language subtleties, which is essential for practical applications where precision and contextual accuracy are key.</p>
    </div>
    <div>
        <h4>The Importance of Rigorous Testing</h4>
        <p>Testing with linguistically complex sentences is vital. It ensures the model can handle various linguistic challenges, an important aspect of deploying models in real-world scenarios.</p>
    </div>
</details>
<br/>

In [10]:
# Load our saved model as a transformers pipeline and validate the performance for a simple translation task
translation_pipeline = mlflow.transformers.load_model(model_info.model_uri)
response = translation_pipeline("I have heard that Nice is nice this time of year.")

print(response)

2023/11/30 12:00:45 INFO mlflow.transformers: 'runs:/2357c12ca17a4f328b2f72cbb7d70343/french_translator' resolved as 'file:///Users/benjamin.wilson/repos/mlflow-fork/mlflow/docs/source/llms/transformers/tutorials/translation/mlruns/996217394074032926/2357c12ca17a4f328b2f72cbb7d70343/artifacts/french_translator'


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

[{'translation_text': "J'ai entendu que Nice est bien cette période de l'année."}]


### Assessing the Reconstructed Pipeline's Translation

We now evaluate the performance of a pipeline reconstructed from loaded components.

<details>
    <div>
        <h4>Reconstruction and Testing</h4>
        <p>Using the dictionary of loaded components, we successfully reconstruct a new translation pipeline. This step is essential to confirm that our logged and retrieved components function cohesively when reassembled.</p>
    </div>
    <div>
        <h4>Translation Quality</h4>
        <p>The reconstructed pipeline adeptly translates English into French, maintaining both the syntactic accuracy and semantic coherence of the original sentence. This reflects the Transformers library's ability to simplify the utilization of complex deep learning models.</p>
    </div>
    <div>
        <h4>Verifying Model Integrity</h4>
        <p>This test is key in verifying that the saved model and its components are not only retrievable but also function effectively post-deployment. It ensures the continued integrity and performance of the model in practical applications.</p>
    </div>
</details>

In [11]:
# Verify that the components that we loaded can be constructed into a pipeline manually
reconstructed_pipeline = transformers.pipeline(**translation_components)

reconstructed_response = reconstructed_pipeline(
    "transformers makes using Deep Learning models easy and fun!"
)

print(reconstructed_response)

[{'translation_text': "transformers simplifie l'utilisation des modèles de l'apprentissage profonde!"}]


### Direct Utilization of Model Components

Explore the granular control over individual model components for custom translation processes.

<details>
    <div>
        <h4>Component Interaction</h4>
        <p>Interacting with the model's individual components offers a deeper level of customization. This approach is particularly beneficial when integrating the model into larger systems or when specific manipulations of inputs and outputs are required.</p>
    </div>
    <div>
        <h4>Insight into Model Structure</h4>
        <p>Examining the keys of the `translation_components` dictionary reveals the structure and components of our model. This includes the task, device mapping, core model, tokenizer, and framework information, each crucial for the translation process.</p>
    </div>
    <div>
        <h4>Benefits of Component-Level Control</h4>
        <p>Utilizing components individually allows for precise adjustments and custom integrations. It's an effective way to tailor the translation process to specific needs, ensuring more control over the model's behavior and output.</p>
    </div>
</details>

In [12]:
# View the components that were saved with our model
translation_components.keys()

dict_keys(['task', 'device_map', 'model', 'tokenizer', 'framework'])

### Advanced Usage: Direct Interaction with Model Components

Direct interaction with a model's components offers flexibility and control for advanced use cases in translation.

<details>
    <div>
        <p>Using the model and tokenizer components directly, as opposed to the higher-level pipeline, allows for:</p>
        <ul>
            <li>Customization of the tokenization process.</li>
            <li>Specific tensor handling, including device specification (CPU, GPU, MPS, etc.).</li>
            <li>Generation and adjustment of predictions on-the-fly.</li>
            <li>Decoding outputs with options for post-processing.</li>
        </ul>
        <p>This approach provides granular control, enabling interventions in the model's operations, such as dynamic input adjustments or output post-processing. However, it also increases complexity, requiring a deeper understanding of the model and tokenizer and the management of more code.</p>
        <p>Opting for direct interaction over the pipeline means balancing ease of use against the level of control required for your application. It's a critical decision, especially for advanced scenarios demanding precise manipulation of the translation process.</p>
    </div>
</details>

In [13]:
# Access the individual components from the components dictionary
tokenizer = translation_components["tokenizer"]
model = translation_components["model"]

query = "Translate to French: Liberty, equality, fraternity, or death."

# This notebook was run on a Mac laptop, so we'll send the output tensor to the "mps" device.
# If you're running this on a different system, ensure that you're sending the tensor output to the appropriate device to ensure that
# the model is able to read it from memory.
inputs = tokenizer.encode(query, return_tensors="pt").to("mps")
outputs = model.generate(inputs).to("mps")
result = tokenizer.decode(outputs[0])

# Since we're not using a pipeline here, we need to modify the output slightly to get only the translated text.
print(result.replace("<pad> ", "\n").replace("</s>", ""))


La liberté, l'égalité, la fraternité ou la mort.


### Tutorial Recap

This tutorial provided insights into combining MLflow with advanced language translation models, emphasizing streamlined model management and deployment.

<details>
    <div>
        <p>We explored several key aspects:</p>
        <ul>
            <li>Setting up and testing a translation pipeline using Transformers.</li>
            <li>Logging the model and its configurations to MLflow for effective versioning and tracking.</li>
            <li>Inferring and examining the model's signature for ensuring input and output consistency.</li>
            <li>Interacting with logged model components for enhanced flexibility in deployment.</li>
            <li>Discussing the nuances of language translation and the role of context in achieving accurate results.</li>
        </ul>
        <h3>The Power of MLflow and Model Metadata</h3>
        <p>MLflow's integration proved instrumental in managing and deploying the translation model. The tutorial highlighted how MLflow's metadata, including the model's signature and flavors, aids in consistent and reliable deployment, catering to production needs.</p>
        <h3>Reflection on the Translation Output</h3>
        <p>The final translation output, while not exact, captured the essence of the iconic French motto, highlighting the model's effectiveness and the importance of contextual understanding in translations. Further exploration on the cultural significance of the phrase can be found on its <a href="https://en.wikipedia.org/wiki/Libert%C3%A9,_%C3%A9galit%C3%A9,_fraternit%C3%A9">Wikipedia page</a>.</p>
        <h3>Conclusion</h3>
        <p>The combination of MLflow and advanced language models like Transformers offers a powerful approach to deploying sophisticated AI solutions. This tutorial aimed to empower your journey in machine learning, whether in translation tasks or other ML applications.</p>
    </div>
</details>